### 활성화 함수
자주 사용하는 활성화 함수로는 step function와 sigmoid function이 있다.

- step function : x = 0을 기준으로 y의 값을 전환한다. x < 0 일때는 y=0, x > 1일 때는 y=1이다.
- sigmoid function : 0~1사이의 값을 매끄러운 곡선으로 표현한다. 식 : y = 1 / (1 + exp(-x))

### 선형, 비선형
선형은 말 그대로 직선 1개로 표현할 수 있는 함수, 비선형의 경우 그러지 못해 곡선이 필요한 함수를 의미한다. 

step function과 sigmoid function 둘 다 비선형 함수로 분류되는데, 신경망의 활성화 함수로는 비선형 함수를 택하는게 좋다. 선형 함수를 채택하는 경우 신경망의 층을 깊게 하는 의미가 사라지며 층을 아무리 깊게 해도 은닉층이 없는 네트워크로 똑같은 기능을 할 수 있다는 문제점이 있기 때문이다.

선형함수인 h(x) = cx 라는 활성화함수를 사용하는 3층 네트워크를 예시로 살펴보면, y(x) = h(h(h(x)))가 되며, 이 계산은 y = cccx 처럼 곱셈을 세 번 수행하지만, 이는 사실 y = ax와 같은 식이다. a = c<sup>3</sup>이라고만 하면 끝이기 때문이다. 즉, 은닉층이 없는 네트워크가 되는 셈이다.

따라서 층을 쌓는 혜택을 누리고 싶다면 활성화 함수로는 비선형 함수를 택하는 편이 좋다.

### numpy의 행렬곱을 이용한 3층 신경망 구현하기

In [1]:
import numpy as np
# 편향값의 경우 입력층이 1인 노드를 추가하여 가중치화 시킨다.

# 활성화 함수 정의(입력층, 은닉층) : 시그모이드 함수
def sigmoid(x):
    return 1/(1+np.exp(-x))
    
# 활성화 함수 정의(출력층) : 항등함수
def identity_function(x):
    return x
    
# 순전파 계산 시작 (입력층 -> 은닉1층)
X = np.array([1.0, 0.5])
W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
B1 = np.array([0.1, 0.2, 0.3])
W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
B2 = np.array([0.1, 0.2])

print(W1.shape)
print(X.shape)
print(B1.shape)

# 계산된 과정 : X[0]*W1[0][0] + X[1]*W[1][0] + B1[0], X[0]*W1[0][1] + X[1]*W[1][1] + B1[1], X[0]*W1[0][2] + X[1]*W[1][2] + B1[2]
A1 = np.dot(X, W1) + B1
print(A1)
Z1 = sigmoid(A1)
print(Z1)

# (은닉1층 -> 은닉2층)
# Z1 = 입력층 -> 은닉1층의 가중합에 활성화 함수를 적용시켜 낸 결과
A2 = np.dot(Z1, W2) + B2
Z2 = sigmoid(A2)
print(A2)
print(Z2)

W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
B3 = np.array([0.1, 0.2])
A3 = np.dot(Z2, W3) + B3
Y = identity_function(A3)
print(A3)
print(Y)

# 이 과정은 총 입력층, 은닉1층, 은닉2층, 출력층으로 총 4개의 층으로 구성되지만 가중치를 가지는 층은 3개의 층뿐이기 때문에 3층 신경망이라고 한다.

(2, 3)
(2,)
(3,)
[0.3 0.7 1.1]
[0.57444252 0.66818777 0.75026011]
[0.51615984 1.21402696]
[0.62624937 0.7710107 ]
[0.31682708 0.69627909]
[0.31682708 0.69627909]


### 출력층 설계하기

신경망은 분류와 회귀 모두에 이용할 수 있습니다만, 어떤 문제를 다루냐에 따라서 출력층에서 사용하는 활성화함수가 달라집니다. 일반적으로 회귀에는 항등함수를, 분류에는 소프트맥스 함수를 사용합니다.

항등함수는 위에서 본 함수 그대로, 입력 신호가 그대로 출력됩니다. 소프트맥스의 경우 아래의 식으로 나타낼 수 있습니다.

y<sub>k</sub> = exp(a<sub>k</sub>) / sum(exp(a<sub>i</sub>)), (i = 1, 2, 3, ..., n)

이 때 n은 출력층의 노드 수, y<sub>k</sub>는 그 중 k번째 출력임을 뜻합니다. 소프트맥스 함수의 분자는 입력 신호 a<sub>k</sub>의 지수함수, 분모는 모든 입력 신호의 지수 함수의 합으로 구성됩니다.
그림으로 살펴보면 아래와 같습니다.
<p align=center><img src="../screenshots/softmax.png"></img></p>

소프트맥스의 출력은 모든 입력 신호로부터 화살표를 받습니다. 분모에서 볼 수 있듯, 출력층의 각 노드가 모든 입력 신호에서 영향을 받기 때문입니다.

In [2]:
# softmax function 구현하기
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

print(softmax([0.3, 2.9, 6.3]))

[0.00239296 0.03221818 0.96538886]


### softmax 함수 구현 시 유의사항
바로 over flow 문제이다. 소프트 맥스 함수는 지수 함수를 사용하는데, 이는 쉽게 아주 큰 값을 내뱉는다. e<sup>10</sup>만 해도 20,000이 넘고 e<sup>100</sup>은 0이 40개가 넘는 큰 수이며, e<sup>1000</sup>은 무한대를 의미하는 inf를 담고있다. 그리고 이런 큰 값끼리 나누면 결과 수치가 '불안정'해진다.

따라서 위의 수식에 분자, 분모에 특정 값을 곱해주고(분자와 분모에 같은 수를 곱하기에 전 식과 동일하다.) 로그의 밑변환 공식을 사용한다.

In [4]:
# 개선한 softmax function
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # over flow error 대비
    sum_exp_a = np.sum(exp_a)
    
    return exp_a / sum_exp_a

print(softmax([0.3, 2.9, 6.3]))

[0.00239296 0.03221818 0.96538886]


### Softmax function의 특징
위의 출력 결과를 확인해보면 소프트맥스 함수의 출력값은 0~1사이의 값이고, 출력의 총합은 1이다.

이러한 성질 덕분에 소프트맥스 함수의 출력을 `확률`로 해석할 수 있다. 위의 경우, `a[0]`의 경우 0.002%, `a[1]`의 경우 0.03%, `a[2]`의 경우 0.96%로 a[2]가 확률이 가장 높으니, 답은 3번째 클래스이다. 라고 답할 수 있겠다.

즉, 소프트맥스 함수를 이용함으로서 문제를 확률적으로 대응할 수 있게 되는 것이다.